In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"theksg9","key":"5783530dd4c30706555b3c3f51310057"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

100% 2.69G/2.70G [00:45<00:00, 63.9MB/s]
100% 2.70G/2.70G [00:45<00:00, 63.7MB/s]


In [ ]:
from zipfile import ZipFile
file_name="/content/new-plant-diseases-dataset.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()

In [ ]:
!mkdir dataset/maize dataset/maize/train dataset/maize/test

In [ ]:
!mv /content/dataset/dataset/valid/Corn* /content/dataset/maize/test
!mv /content/dataset/dataset/train/Corn* /content/dataset/maize/train

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale = 1./255,
	      rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        brightness_range=[0.7,1.3],
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
        )

train_set=train_datagen.flow_from_directory(
    '/content/dataset/maize/train',
    target_size=(64,64),
    batch_size=64,
    class_mode='categorical'
)

test_datagen=ImageDataGenerator(rescale=1./255)

test_set=test_datagen.flow_from_directory(
    '/content/dataset/maize/test',
    batch_size=16,
    target_size=(64,64),
    class_mode='categorical'
)

Found 7316 images belonging to 4 classes.
Found 1829 images belonging to 4 classes.


In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
classifier=Sequential()
classifier.add(Convolution2D(16,(3,3),activation='relu',input_shape=(64,64,3)))
classifier.add(MaxPooling2D())
classifier.add(Convolution2D(32,(3,3),activation='relu'))
classifier.add(MaxPooling2D())
classifier.add(Flatten())
classifier.add(Dense(256,activation='relu'))
classifier.add(Dense(4,activation='softmax'))

In [ ]:
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
classifications=classifier.fit_generator(
    train_set,
    steps_per_epoch=7316/64,
    epochs=15,
    validation_data=test_set,
    validation_steps=1829/16,
    
    verbose=2
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
114/114 - 22s - loss: 0.8352 - accuracy: 0.6372 - val_loss: 0.4988 - val_accuracy: 0.8043
Epoch 2/15
114/114 - 19s - loss: 0.4595 - accuracy: 0.8088 - val_loss: 0.7447 - val_accuracy: 0.7053
Epoch 3/15
114/114 - 19s - loss: 0.3720 - accuracy: 0.8461 - val_loss: 0.7227 - val_accuracy: 0.7080
Epoch 4/15
114/114 - 18s - loss: 0.3424 - accuracy: 0.8636 - val_loss: 0.6771 - val_accuracy: 0.7239
Epoch 5/15
114/114 - 19s - loss: 0.3039 - accuracy: 0.8778 - val_loss: 0.4252 - val_accuracy: 0.8130
Epoch 6/15
114/114 - 18s - loss: 0.3029 - accuracy: 0.8771 - val_loss: 0.4331 - val_accuracy: 0.8141
Epoch 7/15
114/114 - 18s - loss: 0.2664 - accuracy: 0.8963 - val_loss: 0.4003 - val_accuracy: 0.8338
Epoch 8/15
114/114 - 19s - loss: 0.2616 - accuracy: 0.8986 - val_loss: 0.6613 - val_accuracy: 0.7387
Epoch 9/15
114/114 - 18s - loss: 0.2649 - accuracy: 0.8960 - val_loss: 0.3944 - val_accuracy: 0.8420
Epoch 10/15
114/114 - 18s - loss: 0.2405 - accuracy: 0.9038 - val_loss: 0.2794 - val_accura

In [ ]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

#uploaded=files.upload()


 
# predicting images
#path='/content/' + fn
img=image.load_img('/content/nb2.gif', target_size=(64,64))

x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
images=images/255.0
classes = classifier.predict(images, batch_size=10)

print(classes[0])
print(classes)
ar=np.array(classes[0]).tolist()
max_value = max(ar)
max_index = ar.index(max_value)

if max_index==0 :
  print("Gray Leaf Spot")
elif max_index==1:
  print("Common Rust")
elif max_index==2:
  print("Northern Leaf Blight")
else:
  print("Healthy")
  
  # classes = model.predict(images, batch_size=10)
  
  # print(classes[0])
  # print(classes)
  # ar=np.array(classes[0]).tolist()
  # max_value = max(ar)
  # max_index = ar.index(max_value)
  
  # if max_index==0 :
  #   print("Gray Leaf Spot")
  # elif max_index==1:
  #   print("Common Rust")
  # elif max_index==2:
  #   print("Northern Leaf Blight")
  # else:
  #   print("Healthy")

[9.9963617e-01 9.0608808e-08 3.6366837e-04 2.9983181e-15]
[[9.9963617e-01 9.0608808e-08 3.6366837e-04 2.9983181e-15]]
Gray Leaf Spot


In [ ]:
classifier.save('/content/drive/MyDrive/models/maizeNew.h5')

In [ ]:
# Improting Image class from PIL module 
from PIL import Image 
import matplotlib.pyplot as plt
  
# Opens a image in RGB mode 
im = Image.open(r"/content/dataset/maize/test/Corn_(maize)___Common_rust_/RS_Rust 1574.JPG") 
  
# Size of the image in pixels (size of orginal image) 
# (This is not mandatory) 
width, height = im.size 
  
# Setting the points for cropped image 
left = width/4
top = height / 4
right = 3*width/4
bottom = 3 * height / 4
  
# Cropped image of above dimension 
# (It will not change orginal image) 
im1 = im.crop((left, top, right, bottom)) 
  
# Shows the image in image viewer 

plt.show(im1)

In [ ]:
import glob
files=glob.glob("/content/dataset/maize/train/Corn_(maize)___Common_rust_/*.*")


In [ ]:
len(files)

1907

In [ ]:
from PIL import Image
c=0
import os
path="/content/dataset/maize/train/Corn_(maize)___Common_rust_/"
for f in files:
  im = Image.open(f)
  width, height = im.size
  left = width/5
  top = height/5
  right = 4*width/5
  bottom =  4*height/5
  im1 = im.crop((left, top, right, bottom))
  im1.save(path+str(c)+".jpg")
  c=c+1
  os.remove(f)

